In [4]:
from Parameters import ModelParams
from Mass_Loss_Class import MassLoss
from Fractionation_Class import Fractionation
from Model_Data_Loader import ModelDataLoader
from Misc_Class import Misc
from Star_Parameters import StarParams

import numpy as np
import pandas as pd
import json

In [5]:
%matplotlib widget

Unique Teq values found: [ 300. 1000.]

Processing group with Teq = 300.0 K

 Running model with FEUV = 0.20472836758586999 erg/cm^2/s
Converged in 2 iterations for planet 2.0 M_earth.
Final mmw_outflow = 1.0
Planet with mass=2.00 M_earth results: Mdot = 32432.768644347114
Planet with mass=2.00 M_earth results: phi_O = 1.5677376539481684e-132, phi_H = 1.5519445967462713e-15, x_O = 4.040705208768427e-117

Converged in 2 iterations for planet 2.5 M_earth.
Final mmw_outflow = 1.0
Planet with mass=2.48 M_earth results: Mdot = 25629.280158396392
Planet with mass=2.48 M_earth results: phi_O = 3.305773789330658e-135, phi_H = 1.2446651071171195e-15, x_O = 1.0623817669276383e-119

Converged in 2 iterations for planet 3.0 M_earth.
Final mmw_outflow = 1.0
Planet with mass=3.02 M_earth results: Mdot = 21235.899428083576
Planet with mass=3.02 M_earth results: phi_O = 1.721247914386627e-137, phi_H = 1.0237786814538794e-15, x_O = 6.725078166082787e-122

Converged in 2 iterations for planet 3.6 M_eart

In [ ]:
# def main():

# ---------- Initialize parameters and classes ----------
params = ModelParams()
mass_loss = MassLoss(params)
fractionation = Fractionation(params)
star_params = StarParams(params)
misc = Misc(params)
loader = ModelDataLoader('/Users/mvalatsou/PhD/Repos/MR_perplex/OUTPUT/', params)

# ---------- Load data ----------
### option 1
mass, radius, Teq = loader.load_single_ddat_file('3HHe_50H2O_subNeptune.ddat')

### option 2
# with open("planet_params.json", "r") as file:
#     planet_params = json.load(file)
# selected_planet = "2025 HAT-P-67 b" # change the planet here

# if selected_planet in planet_params:
#     new_FEUV = planet_params[selected_planet]["FEUV"]
#     params.update_param("FEUV", new_FEUV)
#     print(f"Updated FEUV for {selected_planet}: {params.get_param('FEUV'):.2e} erg cm-2 s-1")
# else:
#     print(f"Make sure the planet you selected exists in the list! Check the spelling.")

# mearth = params.mearth
# rearth = params.rearth
# radius = np.array([planet_params[selected_planet]["radius"] * rearth]) # cm
# mass = np.array([planet_params[selected_planet]["mass"] * mearth]) # g
# Teq = np.array([planet_params[selected_planet]["teq"]]) # K

# ---------- Find unique temperatures ----------
unique_Teqs = np.unique(Teq)
print("Unique Teq values found:", unique_Teqs)

# ---------- Range of EUV flux values ----------
all_flux_results = []

for Teq_val in unique_Teqs:
    star_params.update_param("Teq", Teq_val)
    flux_range = star_params.get_FEUV_range_any_age()
    # flux_range = [params.get_param('FEUV')] # if planet is selected from json file

    mask = np.isclose(Teq, Teq_val)
    mass_group = mass[mask]
    radius_group = radius[mask]
    teq_group = Teq[mask]
    print(f"\nProcessing group with Teq = {Teq_val} K")

    # ---------- Execute ----------
    for flux in flux_range:
        print(f"\n Running model with FEUV = {flux} erg/cm^2/s")
        params.update_param('FEUV', flux)

        # ---------- Mass loss model ----------
        mass_loss_results = mass_loss.compute_mass_loss_parameters(mass_group, radius_group, teq_group)
        # mdot = mass_loss_results[0]['Mdot']
        # print(f"Mdot =", mdot)
        
        results_for_this_flux = {'FEUV': flux, 'Teq': Teq_val, 'end_results': mass_loss_results}
        
        # ---------- Fractionation model ----------
        ### option 1 - without self consistent feedback loop
        # fractionation_results = fractionation.execute_fractionation(mass_loss_results, misc)
        # results_for_this_flux = {'FEUV': flux, 'Teq': Teq_val, 'end_results': fractionation_results}
        
        ### option 2 - with feedback loop
        fractionation_results = fractionation.execute_self_consistent_fractionation(mass_loss_results, mass_loss, misc, params)
        results_for_this_flux = {'FEUV': flux, 'Teq': Teq_val, 'end_results': fractionation_results}
        
        all_flux_results.append(results_for_this_flux)

In [6]:
# save data for plotting
rows = []

for flux_data in all_flux_results:
    flux = flux_data['FEUV']
    for res in flux_data['end_results']:
        record = {'FEUV': flux}
        record.update(res)
        rows.append(record)

df = pd.DataFrame(rows)
df.to_csv('/Users/mvalatsou/PhD/Repos/MR_perplex/OUTPUT/3HHe_50H2O_subNeptune.csv', index=False)